### 这里，使用 set 每次运行不一致，导致load_model为初始状态，前面训练无效，需要修改

https://machinelearningmastery.com/check-point-deep-learning-models-keras/

https://github.com/keras-team/keras/issues/2378

In [19]:
t_names = ['sam', 'proba']
p_names = ['nsjvn', 'fnosfd']
j = list(set("".join(t_names) + "".join(p_names)))
print(j)
j.sort()
print(j)

['j', 'p', 'n', 'm', 'f', 'a', 'v', 'o', 'd', 'r', 'b', 's']
['a', 'b', 'd', 'f', 'j', 'm', 'n', 'o', 'p', 'r', 's', 'v']


In [10]:
#import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.models import model_from_json
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

import numpy as np
import sys
import os

#from keras.backend import manual_variable_initialization 
#manual_variable_initialization(True)

from keras import backend as K
K.clear_session()

#import keras
#keras.__version__
np.random.seed(42)

def preprocess():
    print("loading data...")
    with open("male.txt") as f:
        m_names = f.read().splitlines()
    
    with open("female.txt") as f:
        f_names = f.read().splitlines()
        
    fs = set(f_names)
    ms = set(m_names)
    bi_names = fs & ms  # unisex names 
    
    m_names = ms - bi_names
    #print(len(m_names))
    
    f_names = fs - bi_names
    #print(len(f_names))
    
    #bi_names = [bi_name.lower() for bi_name in bi_names]
    m_names = [m_name.lower() for m_name in m_names]
    f_names = [f_name.lower() for f_name in f_names]

    # only male and only female names
    n_total = len(m_names) + len(f_names)
    
    max_len = max(len(max(m_names , key=len)), len(max(f_names , key=len)))
    
    chars = set("".join(m_names) + "".join(f_names))
    n_chars = len(chars)
    
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))
    
    print('Corpus...')
    print("total entries: " , n_total)
    print("max_len: " , max_len)
    print('total chars:', n_chars)
    
    X_m = name2vec(m_names, len(m_names), max_len, n_chars, char_indices)
    X_f = name2vec(f_names, len(f_names), max_len, n_chars, char_indices)
    
    y_m = np.ones((len(m_names) , 1))
    y_f = np.zeros((len(f_names) , 1))
    
    X = np.concatenate((X_m, X_f), axis=0)
    y = np.concatenate((y_m, y_f), axis=0)

    return X, y, max_len, n_chars, char_indices


def name2vec(names, n_instances, max_len, n_chars, char_indices):
    X = np.zeros((n_instances, max_len, n_chars))
    for i, name in enumerate(names):
        for t, char in enumerate(name):
            X[i, t, char_indices[char]] = 1.0
    return X


def train_test_split(X, y, n_train=6000):
    n_total = X.shape[0]
    idx = np.random.permutation(n_total)
    
    X_train = X[idx][:n_train]
    y_train = y[idx][:n_train]
    
    X_test = X[idx][n_train:]
    y_test = y[idx][n_train:]

    return X_train, y_train, X_test, y_test


def build(max_len, n_chars):

    print('Build model...')
    model = Sequential()
    model.add(layers.LSTM(128, return_sequences=True, dropout=0.2, input_shape=(max_len, n_chars)))
    #model.add(layers.Dropout(0.2))
    model.add(layers.LSTM(128, return_sequences=False, dropout=0.2))
    #model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model


'''
history = model.fit(X_train, y_train,
                    epochs=n_epochs,
                    batch_size=128,
                    validation_split=0.2)
'''

'''
from matplotlib import pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()
'''
def predict_gender(names, pred):
    print('Model predict...')
    p_male = 0
    p_female = 0
    for i in range(pred.shape[0]):
        if pred[i] > 0.5:
            p_male += 1
#print('%s is Male'%names[i])
        else:
            p_female += 1
#print('%s is Female'%names[i])
    print(p_male, p_female)


In [11]:
X, y, max_len, n_chars, char_indices = preprocess()
X_train, y_train, X_test, y_test = train_test_split(X, y)

#filepath="weights.best.hdf5"
filepath = 'weights.h5'
#filepath = 'gender_model_weights.h5'
model = build(max_len, n_chars)

model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
n_epochs = 5
history = model.fit(X_train, y_train,
                        epochs=n_epochs,
                        batch_size=64,
                        validation_split=0.2)
print('after training')
results = model.evaluate(X_test, y_test)
print('test :',results)

loading data...
Corpus...
total entries:  7214
max_len:  15
total chars: 29
Build model...
Train on 4800 samples, validate on 1200 samples
Epoch 1/5
4800/4800 [==============================] - 7s 1ms/step - loss: 0.6043 - acc: 0.6773 - val_loss: 0.5285 - val_acc: 0.7442
Epoch 2/5
4800/4800 [==============================] - 6s 1ms/step - loss: 0.5381 - acc: 0.7342 - val_loss: 0.4981 - val_acc: 0.7542
Epoch 3/5
4800/4800 [==============================] - 6s 1ms/step - loss: 0.5287 - acc: 0.7400 - val_loss: 0.5138 - val_acc: 0.7292
Epoch 4/5
4800/4800 [==============================] - 6s 1ms/step - loss: 0.5146 - acc: 0.7467 - val_loss: 0.5232 - val_acc: 0.7300
Epoch 5/5
4800/4800 [==============================] - 6s 1ms/step - loss: 0.5180 - acc: 0.7496 - val_loss: 0.4875 - val_acc: 0.7567
after training
1214/1214 [==============================] - 1s 417us/step
test : [0.4699836440593053, 0.7883031297554883]


In [12]:
origin_names = []
pred = model.predict(X_test)
predict_gender(origin_names, pred)

Model predict...
283 931


In [13]:
model.save(filepath)
#model.save(filepath)
del model

In [14]:
#model

In [15]:
model = load_model(filepath)
print('successfully retriving model...')

successfully retriving model...


In [17]:
results = model.evaluate(X_test, y_test)
print('test :',results)


1214/1214 [==============================] - 1s 455us/step
test : [0.4699836440593053, 0.7883031297554883]


In [18]:
sum(y_test == 0.0)

array([804])